In [6]:
import numpy as np
import math
import cmath
import matplotlib.pyplot as plt

In [7]:
class ACModel:
    def __init__(self, Ro, Rm, Ri, L, C1, C2, EMF, circuitnum):
        self.Ro = Ro
        self.Rm = Rm
        self.Ri = Ri
        self.L = L
        self.C1 = C1
        self.C2 = C2
        self.EMF = EMF
        self.circuitnum = circuitnum
        
    
    #forward euler timestepped to determine next function value using current funcition value, derivative
    def timestepfunc(self, f0, derf, dt):
        f = f0 + derf*dt
        return f
    
    def phaseangle(self, a):
        return cmath.phase(a)
    
    #assumes inductors = circuit break and capacitor = wire at t = 0 to solve for intial currents
    def I0(self):
        I_0 = np.zeros([2*self.circuitnum+1, 1], dtype = 'double') #array to hold the various currents' initial values
        R_0 = np.zeros([2*self.circuitnum+1, 1], dtype = 'double') # 'weights' defined during R0_eq calculations to detemine initial current split
        sys = np.zeros([self.circuitnum+1, self.circuitnum+1], dtype = 'double') #will hold linear equations determining current split
        sys2 = np.zeros([3*self.circuitnum+1, 3*self.circuitnum+1], dtype='double') #columns: I0, I1, I2, I3, I4, I5, I6, I'1, I'3, I'5
        solcol = np.zeros([self.circuitnum+1, 1], dtype = 'double') #used as solution matrix for sys
        soln2 = np.zeros([3*self.circuitnum+1], dtype = 'double')#solution matrix for sys2
        R_eq = self.Rm
        
        for n in range(self.circuitnum):
            R_eq += 2*(self.Ri+self.Ro)
            R_0[2*(self.circuitnum - n)] = R_eq #only need circuitnum-1 terms, last term generated is a throwaway
            R_eq = 1/((1/R_eq)+1/self.Rm)
       
        I_max = self.EMF/R_eq
        print("\nEquivalent resistance initially: ")
        print(R_eq)
        solcol[self.circuitnum] = I_max
        
        for i in range(self.circuitnum):
            sys[self.circuitnum - 1 - i][i] = -1
            for j in range(i):
                sys[i-1][j-i] = (R_0[2*(self.circuitnum - i+1)])/self.Rm
                
        for x in range(self.circuitnum+1):
            sys[self.circuitnum-1][x] = -1.0
            sys[self.circuitnum-1][0] = 1.0
        
        sys[self.circuitnum][0] = 1.0    
        solution = np.linalg.solve(sys, solcol)    
    
        for n in range(self.circuitnum+1):
            I_0[2*n-1] = 0.0
            I_0[2*n] = solution[n]

        print("\nInitial Currents: ")
        print(I_0)
        return I_0
    

In [8]:
model = ACModel(1, 3.3, 2.2, 1e-5, 1e-5, 1e-6, 9, 3)
model.I0()
#a = complex(1, 2)
#model.phaseangle(a)



Equivalent resistance initially: 
2.4003428150439428

Initial Currents: 
[[3.74946443]
 [0.        ]
 [2.73212192]
 [0.        ]
 [0.75909403]
 [0.        ]
 [0.25824848]]


array([[3.74946443],
       [0.        ],
       [2.73212192],
       [0.        ],
       [0.75909403],
       [0.        ],
       [0.25824848]])